### 오염

In [1]:
import json
import random
import os
from tqdm import tqdm
from glob import glob

In [2]:
# --- 설정 ---
INPUT_FILE = "./results/cleaned/bert_train.json"
OUTPUT_DIR = "./results/corrupted"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# [cite_start]특수문자 풀 [cite: 12, 19]
SPECIAL_CHARS = ['!', '@', '#', '$', '%', '^', '&', '*']

def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# --- 1. Deletion Noise Logic (삭제) ---
def get_deletion_noise(text, answer_text, answer_start, ratio=0.15):
    # 정답 텍스트 파악 (원본 유지)
    answer_end = answer_start + len(answer_text)
    ans_text_original = text[answer_start:answer_end]
    
    # 정답 앞/뒤 텍스트 분리
    pre_text = text[:answer_start]
    post_text = text[answer_end:]
    
    # 공백 기준 토큰화
    pre_tokens = pre_text.split(' ')
    post_tokens = post_text.split(' ')
    
    def delete_tokens(tokens, r):
        if not tokens: return []
        # 빈 문자열 토큰은 인덱스 계산에서 제외
        valid_indices = [i for i, t in enumerate(tokens) if t.strip()]
        n_delete = int(len(valid_indices) * r)
        
        if n_delete == 0: return tokens
        
        indices_to_delete = set(random.sample(valid_indices, n_delete))
        return [t for i, t in enumerate(tokens) if i not in indices_to_delete]

    new_pre_tokens = delete_tokens(pre_tokens, ratio)
    new_post_tokens = delete_tokens(post_tokens, ratio)
    
    # 재조립
    new_pre_text = ' '.join(new_pre_tokens)
    # 앞 문장이 줄어들었거나 유지될 때, 원본이 공백으로 끝났다면 공백 유지
    if len(new_pre_text) > 0 and not new_pre_text.endswith(' ') and pre_text.endswith(' '):
        new_pre_text += ' '
        
    # [핵심] 정답 시작 위치 갱신: 변형된 앞부분 텍스트의 길이로 재설정
    new_answer_start = len(new_pre_text)
    
    new_post_text = ' '.join(new_post_tokens)
    if len(new_post_text) > 0 and not new_post_text.startswith(' ') and post_text.startswith(' '):
        new_post_text = ' ' + new_post_text
        
    new_context = new_pre_text + ans_text_original + new_post_text
    
    # 수정된 부분: 반환값 3개로 맞춤 (Context, Start Index, Answer Text)
    return new_context, new_answer_start, ans_text_original

# --- 2. Context Insertion Logic (지문 오염) ---
def get_context_insertion_noise(text, answer_text, answer_start, ratio=0.3):
    answer_end = answer_start + len(answer_text)
    
    # 리스트로 변환하여 문자 삽입
    pre_text = list(text[:answer_start])
    ans_text_original = text[answer_start:answer_end]
    post_text = list(text[answer_end:])
    
    def insert_noise(char_list, r):
        if not char_list: return ""
        n_insert = int(len(char_list) * r)
        for _ in range(n_insert):
            pos = random.randint(0, len(char_list)) # 랜덤 위치 삽입
            char = random.choice(SPECIAL_CHARS)
            char_list.insert(pos, char)
        return "".join(char_list)

    new_pre_text = insert_noise(pre_text, ratio)
    new_post_text = insert_noise(post_text, ratio)
    
    # [핵심] 정답 시작 위치 갱신: 늘어난 앞부분 텍스트의 길이로 재설정
    new_answer_start = len(new_pre_text)
    new_context = new_pre_text + ans_text_original + new_post_text
    
    return new_context, new_answer_start, ans_text_original

# --- 3. Answer Insertion Logic (정답 오염) ---
def get_answer_insertion_noise(text, answer_text, answer_start, ratio=0.3):
    answer_end = answer_start + len(answer_text)
    
    pre_text = text[:answer_start]
    ans_chars = list(text[answer_start:answer_end])
    post_text = text[answer_end:]
    
    n_insert = int(len(ans_chars) * ratio)
    n_insert = max(1, n_insert)
    
    for _ in range(n_insert):
        pos = random.randint(1, len(ans_chars)-1) if len(ans_chars) > 1 else 0
        char = random.choice(SPECIAL_CHARS)
        ans_chars.insert(pos, char)
        
    new_ans_text = "".join(ans_chars)
    new_context = pre_text + new_ans_text + post_text
    
    # 정답 내부가 변한 것이므로 시작 인덱스(answer_start)는 변하지 않음
    return new_context, answer_start, new_ans_text

# --- 통합 생성 함수 ---
def generate_all_datasets():
    print(f"🔄 Loading input: {INPUT_FILE}")
    try:
        source_data = load_json(INPUT_FILE)
    except FileNotFoundError:
        print("❌ Input file not found. Please run the preprocessing step first.")
        return

    datasets = {
        "baseline":      {"version": "KorQuAD_v1.0_baseline", "data": []},
        "deletion":      {"version": "KorQuAD_v1.0_deletion", "data": []},
        "insertion_que": {"version": "KorQuAD_v1.0_insertion_que", "data": []},
        "insertion_ans": {"version": "KorQuAD_v1.0_insertion_ans", "data": []}
    }
    
    print("⚡ Processing all noise types...")
    
    for doc in tqdm(source_data['data']):
        doc_structs = {
            k: {"title": doc['title'], "paragraphs": []} for k in datasets.keys()
        }
        
        for para in doc['paragraphs']:
            original_context = para['context']
            
            for qa in para['qas']:
                ans = qa['answers'][0]
                a_text = ans['text']
                a_start = ans['answer_start']
                
                # 1. Baseline
                doc_structs["baseline"]["paragraphs"].append({
                    "context": original_context,
                    "qas": [qa]
                })

                # 2. Deletion (반환값 3개 매칭 수정 완료)
                del_ctx, del_start, del_ans = get_deletion_noise(original_context, a_text, a_start, ratio=0.15)
                doc_structs["deletion"]["paragraphs"].append({
                    "context": del_ctx,
                    "qas": [{
                        "id": qa['id'], "question": qa['question'],
                        "answers": [{"text": del_ans, "answer_start": del_start}]
                    }]
                })
                
                # 3. Insertion Que
                ins_q_ctx, ins_q_start, ins_q_ans = get_context_insertion_noise(original_context, a_text, a_start, ratio=0.3)
                doc_structs["insertion_que"]["paragraphs"].append({
                    "context": ins_q_ctx,
                    "qas": [{
                        "id": qa['id'], "question": qa['question'],
                        "answers": [{"text": ins_q_ans, "answer_start": ins_q_start}]
                    }]
                })
                
                # 4. Insertion Ans
                ins_a_ctx, ins_a_start, ins_a_ans = get_answer_insertion_noise(original_context, a_text, a_start, ratio=0.3)
                doc_structs["insertion_ans"]["paragraphs"].append({
                    "context": ins_a_ctx,
                    "qas": [{
                        "id": qa['id'], "question": qa['question'],
                        "answers": [{"text": ins_a_ans, "answer_start": ins_a_start}]
                    }]
                })
        
        # 문서 저장
        for k in datasets.keys():
            if doc_structs[k]["paragraphs"]:
                datasets[k]["data"].append(doc_structs[k])

    print("💾 Saving files...")
    for key, data_content in datasets.items():
        filename = f"train_{key}.json"
        save_path = os.path.join(OUTPUT_DIR, filename)
        save_json(data_content, save_path)
        print(f"   - Saved: {save_path}")

In [4]:
# 실행
generate_all_datasets()

🔄 Loading input: ./results/cleaned/bert_train.json
⚡ Processing all noise types...


100%|█████████████████████████████████████████████████████████████████████████| 175807/175807 [07:17<00:00, 402.14it/s]


💾 Saving files...
   - Saved: ./results/corrupted\train_baseline.json
   - Saved: ./results/corrupted\train_deletion.json
   - Saved: ./results/corrupted\train_insertion_que.json
   - Saved: ./results/corrupted\train_insertion_ans.json


### 검증

In [5]:
# 검증 대상 경로
TARGET_DIR = "./results/corrupted"
BASELINE_FILE = os.path.join(TARGET_DIR, "train_baseline.json")
SPECIAL_CHARS = ['!', '@', '#', '$', '%', '^', '&', '*']

def load_json(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"❌ Load Failed: {filepath} ({e})")
        return None

def build_reference_map(baseline_data):
    """
    원본(Baseline) 데이터를 QA ID 기준으로 빠르게 조회하기 위한 맵 생성
    """
    ref_map = {}
    for doc in baseline_data['data']:
        for para in doc['paragraphs']:
            context = para['context']
            for qa in para['qas']:
                qid = qa['id']
                ans = qa['answers'][0]
                ref_map[qid] = {
                    "context": context,
                    "text": ans['text'],
                    "start": ans['answer_start']
                }
    return ref_map

def verify_dataset(filepath, mode, ref_map):
    filename = os.path.basename(filepath)
    print(f"\n🔎 Verifying: {filename} (Mode: {mode})")
    
    data = load_json(filepath)
    if not data: return

    stats = {
        "total": 0,
        "alignment_ok": 0,
        "alignment_fail": 0,
        "corruption_ok": 0,   # 오염이 의도대로 적용됨 (또는 의도된 스킵)
        "corruption_fail": 0, # 오염이 적용되지 않음 (수치 변화 없음 등)
        "skipped_short": 0,   # [추가] 길이가 짧아 삭제가 생략된 건수
        "errors": []
    }

    # 모든 데이터 순회
    for doc in tqdm(data['data'], desc="Checking samples"):
        for para in doc['paragraphs']:
            context = para['context']
            for qa in para['qas']:
                stats["total"] += 1
                qid = qa['id']
                
                # 현재 데이터의 정답 정보
                curr_ans = qa['answers'][0]
                curr_text = curr_ans['text']
                curr_start = curr_ans['answer_start']

                # 1. Alignment Check (자가 검증)
                # 인덱스로 잘라낸 텍스트가 정답 텍스트와 일치하는지
                extracted = context[curr_start : curr_start + len(curr_text)]
                if extracted == curr_text:
                    stats["alignment_ok"] += 1
                else:
                    stats["alignment_fail"] += 1
                    if len(stats["errors"]) < 3:
                        stats["errors"].append(f"[Align Error] ID: {qid} | Expected: {curr_text} != Extracted: {extracted}")

                # 2. Corruption Check (원본 비교 검증)
                if mode == 'baseline':
                    stats["corruption_ok"] += 1 # Baseline은 검증 대상 아님
                    continue

                if qid not in ref_map:
                    continue
                
                ref = ref_map[qid]
                is_corrupted = False
                
                if mode == 'deletion':
                    # [삭제 검증 A] 원본보다 Context 길이가 짧아졌는지 확인
                    if len(context) < len(ref['context']):
                        is_corrupted = True
                    else:
                        # [삭제 검증 B - 추가된 로직]
                        # 길이가 안 줄었다면, 원본이 너무 짧아서(7단어 미만) 그런 것인지 확인
                        # 생성 코드의 기준: int(word_count * 0.15) == 0 이면 스킵됨
                        ref_word_count = len(ref['context'].split())
                        if ref_word_count < 7:
                            # 7단어 미만이라 삭제 안 된 것이므로 "정상 동작"으로 간주
                            is_corrupted = True 
                            stats["skipped_short"] += 1
                    
                elif mode == 'insertion_que':
                    # [cite_start][지문 오염 검증] Context 길이가 늘어나고 특수문자가 포함 [cite: 12]
                    has_special = any(c in context for c in SPECIAL_CHARS)
                    is_longer = len(context) > len(ref['context'])
                    if has_special and is_longer:
                        is_corrupted = True
                        
                elif mode == 'insertion_ans':
                    # [cite_start][정답 오염 검증] 정답 Text 길이가 늘어나고 특수문자 포함 [cite: 19]
                    has_special = any(c in curr_text for c in SPECIAL_CHARS)
                    is_longer = len(curr_text) > len(ref['text'])
                    if has_special and is_longer:
                        is_corrupted = True
                
                if is_corrupted:
                    stats["corruption_ok"] += 1
                else:
                    stats["corruption_fail"] += 1

    # --- 결과 출력 ---
    print(f"   📊 Results for {filename}")
    print(f"   - Total Samples: {stats['total']}")
    print(f"   - ✅ Alignment OK: {stats['alignment_ok']} ({(stats['alignment_ok']/stats['total'])*100:.1f}%)")
    
    if stats['alignment_fail'] > 0:
        print(f"   - ❌ Alignment Failed: {stats['alignment_fail']}")
        for err in stats['errors']:
            print(f"      -> {err}")
            
    if mode != 'baseline':
        print(f"   - 🧪 Corruption Verified: {stats['corruption_ok']} ({(stats['corruption_ok']/stats['total'])*100:.1f}%)")
        
        # [추가된 출력] 짧은 문장 스킵 통계 표시
        if mode == 'deletion' and stats['skipped_short'] > 0:
             print(f"     (ℹ️ Includes {stats['skipped_short']} samples skipped due to length < 7 words)")

        print(f"   - ⚠️ Unchanged Samples (Fail): {stats['corruption_fail']}")
        if stats['corruption_fail'] > 0:
            print("     (Note: Insertion failed on empty strings or other edge cases)")
    
    print("-" * 50)

# --- 메인 실행 로직 ---
def main():
    if not os.path.exists(BASELINE_FILE):
        print("❌ Baseline file not found. Please run corrupter.ipynb first.")
        return

    print("📥 Loading Baseline Data for comparison...")
    baseline_data = load_json(BASELINE_FILE)
    ref_map = build_reference_map(baseline_data)
    print(f"   Loaded {len(ref_map)} reference QA pairs.\n")

    # glob.glob 사용으로 수정
    files = glob(os.path.join(TARGET_DIR, "train_*.json"))
    files.sort()

    for file_path in files:
        mode = os.path.basename(file_path).replace("train_", "").replace(".json", "")
        verify_dataset(file_path, mode, ref_map)

if __name__ == "__main__":
    main()

📥 Loading Baseline Data for comparison...
   Loaded 292952 reference QA pairs.


🔎 Verifying: train_baseline.json (Mode: baseline)


Checking samples: 100%|████████████████████████████████████████████████████| 175807/175807 [00:00<00:00, 194022.21it/s]


   📊 Results for train_baseline.json
   - Total Samples: 292952
   - ✅ Alignment OK: 292952 (100.0%)
--------------------------------------------------

🔎 Verifying: train_deletion.json (Mode: deletion)


Checking samples: 100%|█████████████████████████████████████████████████████| 175807/175807 [00:01<00:00, 96190.34it/s]


   📊 Results for train_deletion.json
   - Total Samples: 292952
   - ✅ Alignment OK: 292952 (100.0%)
   - 🧪 Corruption Verified: 292941 (100.0%)
     (ℹ️ Includes 6 samples skipped due to length < 7 words)
   - ⚠️ Unchanged Samples (Fail): 11
     (Note: Insertion failed on empty strings or other edge cases)
--------------------------------------------------

🔎 Verifying: train_insertion_ans.json (Mode: insertion_ans)


Checking samples: 100%|█████████████████████████████████████████████████████| 175807/175807 [00:02<00:00, 74103.76it/s]


   📊 Results for train_insertion_ans.json
   - Total Samples: 292952
   - ✅ Alignment OK: 292952 (100.0%)
   - 🧪 Corruption Verified: 292952 (100.0%)
   - ⚠️ Unchanged Samples (Fail): 0
--------------------------------------------------

🔎 Verifying: train_insertion_que.json (Mode: insertion_que)


Checking samples: 100%|█████████████████████████████████████████████████████| 175807/175807 [00:02<00:00, 76371.95it/s]


   📊 Results for train_insertion_que.json
   - Total Samples: 292952
   - ✅ Alignment OK: 292952 (100.0%)
   - 🧪 Corruption Verified: 292952 (100.0%)
   - ⚠️ Unchanged Samples (Fail): 0
--------------------------------------------------
